**Top Tagger Framework**

**Framework version January 2024**
- Log :
    - understood bug in data (the problem was in CRAB not here)
    - added Snapshot 
- Planned update :
    - standalone code, prepare a couple of input parameters and make the code working with a single command
    
    
__________________________________________

**Code**

Folder definition on Tier:
- in the main folder */acagnott/* added folder 'remote_folder_name';
- in "Snapshots" added the folder 'remote_subfolder_name';
- in the subfolder through dask the snapshot will be copied with name "snap_"+label+"_*.root"


Es: se lancio "DataMETA_2018", gli snapshot vengono salvati in ../acagnott/Snapshot/20231229/snap_DataMET_2018_*.root
se viene lanciato "QCD_2018" viene creata la cartella /acagnott/Snapshot/20231229/snap_QCDHT_100to200_2018_*.root e così via per ogni components

---> Viene usato solo il giorno in modo che tutti i sample lanciati lo stesso giorno verranno salvati nella stessa cartella con nomi diversi, visto che vengono lanciati in momenti diversi della giornata lo stesso tipo di job. Forse va modificato il formato se i singoli job iniziano a durare più di un giorno dato che viene comunque lanciato un sampel per volta (in tal caso verrebbero salvati in cartelle diverse. Si potrebbe pensare di mettere la data a mano, cioé invece di usare datetime.now() si potrebbe inserire la data manualmente per fare in modo di mettere tutti i file nella stessa folder. 

In [1]:
from datetime import datetime
sched_port = 22358 #Dask port
nmaxpartition = 100
distributed = False#True#
do_histos = True
hist_folder = "TopTagger_JetMET_selectioncomparison"
do_snapshot = True
remote_subfolder_name = datetime.now().strftime("%Y%m%d") #20231229

in_dataset = [
    # "DataMETA_2018", 
    # "DataMETB_2018", 
    # "DataMETC_2018",
    # "DataMETD_2018",
    # "DataSingleMuA_2018", 
    # "DataSingleMuB_2018", 
    # "DataSingleMuC_2018",
    # "DataSingleMuD_2018",
    
    # "TprimeToTZ_700_2018",
    # "TprimeToTZ_1000_2018", 
    # "TprimeToTZ_1800_2018",
    
    # "QCD_2018",
    # "TT_2018",
    # "ZJetsToNuNu_2018",
    # "WJets_2018",
    
    # "TT_semilep_2018",
    # "TT_hadr_2018",
    # "TT_Mtt700to1000_2018",
    # "TT_Mtt1000toInf_2018",
    # "QCDHT_100to200_2018", "QCDHT_200to300_2018","QCDHT_300to500_2018", "QCDHT_500to700_2018", "QCDHT_700to1000_2018","QCDHT_1000to1500_2018", "QCDHT_1500to2000_2018","QCDHT_2000toInf_2018","TT_hadr_2018", "TT_semilep_2018", "TT_Mtt700to1000_2018", "TT_Mtt1000toInf_2018","ZJetsToNuNu_HT100to200_2018", "ZJetsToNuNu_HT200to400_2018", "ZJetsToNuNu_HT400to600_2018", "ZJetsToNuNu_HT600to800_2018", "ZJetsToNuNu_HT800to1200_2018", "ZJetsToNuNu_HT1200to2500_2018", "ZJetsToNuNu_HT2500toInf_2018", "WJetsHT100to200_2018", "WJetsHT200to400_2018", "WJetsHT400to600_2018", "WJetsHT600to800_2018", "WJetsHT800to1200_2018", "WJetsHT1200to2500_2018", "WJetsHT2500toInf_2018"
    # "QCDHT_100to200_2018", 
    # "QCDHT_200to300_2018",
    # "QCDHT_300to500_2018", 
    # "QCDHT_500to700_2018", 
    # "QCDHT_700to1000_2018", 
    # "QCDHT_1000to1500_2018", 
    # "QCDHT_1500to2000_2018",
    # "QCDHT_2000toInf_2018"
    # "WJetsHT100to200_2018", 
    # "WJetsHT200to400_2018", 
    # "WJetsHT400to600_2018", 
    # "WJetsHT600to800_2018", 
    # "WJetsHT800to1200_2018",
    # "WJetsHT1200to2500_2018", 
    "WJetsHT2500toInf_2018"
]

In [2]:
import ROOT
import os
from utils.samples import *
from utils.variables_tagger import *
import json
import numpy as np
import math
from dask.distributed import Client
import psutil
from psutil import cpu_percent
ROOT.RDF.Experimental.Distributed.open_files_locally = False

os.environ['X509_CERT_DIR'] = "/cvmfs/grid.cern.ch/etc/grid-security/certificates/"
os.environ['X509_USER_PROXY'] = "/tmp/x509up_u0"
print(os.environ.get("X509_USER_PROXY"), os.environ.get("X509_CERT_DIR"))


if distributed:
    nfiles_max = 10000
else:
    nfiles_max = 1  #######

# Cosa aggiungere, modificare la cartella sul tier in questo modo ../acagnott/Snapshot_rdf/*dataset_name*/*data di processamente con orario*/
# insomma come fa crab


if do_histos: print("You are producing histograms")
if do_snapshot: print("You are producing snapshot")

remote_folder_name = "Snapshots"
#output histos folder
folder = "./results/"+hist_folder+"/"
# eos_folder = "/eos/home-a/acagnott/DarkMatter/nosynch/"+hist_folder

if do_snapshot and remote_subfolder_name == datetime.now().strftime("%Y%m%d") and distributed: 
    print("You are naming the tier subfolder using the current day \n")
    print("Snapshots folder name : ~/acagnott/{}/{}".format(remote_folder_name, remote_subfolder_name))
elif do_snapshot and distributed:
    print("You are naming the tier subfolder manually")
    print("Snapshots folder name : ~/acagnott/{}/{}".format(remote_folder_name, remote_subfolder_name))
elif do_snapshot:
    print("You are saving snapshots in local")
    print("folder name : " + folder)


if do_histos : 
    print("local folder histos: {}".format(folder))
    # print("At the end of the job the histos will be transfer to eos storage")
    # print("eos folder : {}".format(eos_folder))
if not os.path.exists(folder):
    os.mkdir(folder)
repohisto = folder+"plots/"
if not os.path.exists(repohisto):
    os.mkdir(repohisto)

Welcome to JupyROOT 6.27/01
/tmp/x509up_u0 /cvmfs/grid.cern.ch/etc/grid-security/certificates/
You are producing histograms
You are producing snapshot
You are saving snapshots in local
folder name : ./results/TopTagger_JetMET_selectioncomparison/
local folder histos: ./results/TopTagger_JetMET_selectioncomparison/


In [3]:
# Creating folders on Tier
if do_snapshot and distributed:
    tier_main_folder = "davs://stwebdav.pi.infn.it:8443/cms/store/user/acagnott/"
    os.popen("davix-mkdir davs://stwebdav.pi.infn.it:8443/cms/store/user/acagnott/{} -E /tmp/x509up_u0 --capath /cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/".format(remote_folder_name))
    os.popen("davix-mkdir davs://stwebdav.pi.infn.it:8443/cms/store/user/acagnott/{}/{} -E /tmp/x509up_u0 --capath /cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/".format(remote_folder_name, remote_subfolder_name))
    
# transfer function for dask worker
def transfer_to_tier(dask_worker):
    import os
    os.popen('for filename in snapTagger_*.root; do davix-put $filename davs://stwebdav.pi.infn.it:8443/cms/store/user/acagnott/{}/{}/$filename -E ./proxy --capath /cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/; done'.format(remote_folder_name, remote_subfolder_name))
    return True #, os.popen("for filename in ./test_*.root; do echo $filename; done").read()

In [4]:
def get_worker_cpu_usage(client):
    workers = list(client.scheduler_info()["workers"].keys())
    cpu_usage = {}

    for worker in workers:
        # Ottieni l'utilizzo della CPU per ogni worker
        cpu_percent = client.run(psutil.cpu_percent)

        # Calcola la media dell'utilizzo della CPU per ogni worker
        avg_cpu_percent = sum(cpu_percent.values()) / len(cpu_percent)
        
        # Memorizza i risultati nell'oggetto cpu_usage
        cpu_usage[worker] = avg_cpu_percent

    return cpu_usage

- Import of utils from variables.py
Cut (if any), Regions, Variables

- syncro between in_dataset and sample_dict (from sample.py) to syncronize labels and ather featurs of the dataset (as sigma if needed)
- import of samples_dict.json to load files list (path to reach them on tier)


In [5]:
# Common cuts over the different regions
cut = requirements # ---> see variables.py

# Regions definition
regions_def = regions # ---> see variables.py
print("Regions to book: ")
for r in regions_def.keys():
    print("  "+r)
    
# Samples dictionary --> to load the files of each samples    
sample_file = open("utils/dict_samples.json", "rb")
samples = json.load(sample_file)
sample_file.close()

# WP top taggers
Top_threshold = {"Resolved" : {'WPloose': "0.24193972", 'WPmedium': "0.5411276", 'WPtight': "0.77197933"},
                 "Mixed"    :{'WPloose': "0.2957885265350342", 'WPmedium': "0.7584613561630249", 'WPtight': "0.9129540324211121"}}
TopMerged_threshold = {'WPloose': "0.85", 'WPmedium': "0.94", 'WPtight': "0.97"}
# Variables definition 
var = vars  # ---> variables.py
var2d = vars2D 

print("Variables for histograms :")
print([v._name for v in var])
print("Variables for histograms 2D :")
print([v._name for v in var2d])

# Alternative definition of truth

print(TruthType)

# Snapshots branches

branches = {"isMC","year","HEMVeto","HT_eventHT", "MET_pt", "MET_phi", "w_nominal",
            "Top_score_Resolved", "Top_pt_Resolved", "Top_eta_Resolved", "Top_phi_Resolved", "Top_mass_Resolved", "Top_idx_Resolved",
            "Top_score_Mixed", "Top_pt_Mixed", "Top_eta_Mixed", "Top_phi_Mixed", "Top_mass_Mixed", "Top_idx_Mixed",
            "Top_idx_Resolved", "Top_idx_Mixed", "NGoodTopMu","TopMerged_bestTopTruth", "TopMerged_bestTopScore",
            "TopMerged_bestTopPt", "TopMerged_bestTopEta", "TopMerged_bestTopPhi", "TopMerged_bestTopMass", "TopMerged_truth_exists",
            # # "BJetTopLep", "BJetTopLep_MCtag", }
            "BJetTopMu"}

for wp in ["WPloose","WPmedium","WPtight"]:
    branches.update(["Top_mass_"+wp, "Top_pt_"+wp, "EventTopCategory_"+wp])
            
for top in topTypes:
    branches.update([
        "nClusterT2Top"+top+"",
        "nClusterT1Top"+top+""])
    for wp in ["WPloose","WPmedium","WPtight"]:
        branches.update(["nTopClusterT2"+top+"Reco_"+wp,
        "nTopClusterT1"+top+"Reco_"+wp])
for t in TruthType.keys():
        top = t.replace("Top","")
        for tr in TruthType[t].keys():
            branches.update(["nTopClusterT2"+top+"MCTag"+tr,
                             "nTopClusterT1"+top+"MCTag"+tr,
                             "TopClusterT2"+top+"MCTagTrueFirstCluster"+tr,
                             "TopClusterT1"+top+"MCTagTrueFirstCluster"+tr,
                             "TopClusterT2"+top+"MCTagFalseFirstCluster"+tr,
                             "TopClusterT1"+top+"MCTagFalseFirstCluster"+tr
                             ])
            for wp in ["WPloose","WPmedium","WPtight"]:
                branches.update(["nTopClusterT2"+top+"TrueReco"+tr+"_"+wp,
                             "nTopClusterT1"+top+"TrueReco"+tr+"_"+wp,
                             "TopClusterT2"+top+"TrueRecoFirstCluster"+tr+"_"+wp,
                             "TopClusterT1"+top+"TrueRecoFirstCluster"+tr+"_"+wp,
                             "TopClusterT2"+top+"FalseRecoFirstCluster"+tr+"_"+wp,
                             "TopClusterT1"+top+"FalseRecoFirstCluster"+tr+"_"+wp])


# Loading datasets choose at the beginning of the notebook
datasets = []
for in_d in in_dataset:
    if not in_d in sample_dict.keys():
        print("Check the in_dataset string... ", sample_dict.keys())
    else : 
        datasets.append(sample_dict[in_d])
print("Datasets to process : ", [d.label for d in datasets])

# Loading the files strings to pass to the DataFrame
chain = {}
ntot_events = {}
for d in datasets:
    if hasattr(d, "components"):
        samples_list = d.components
    else:
        samples_list = [d]
    chain[d.label] = {}
    ntot_events[d.label] = {}
    for s in samples_list:
        if distributed: 
            nfiles = len(samples[d.label][s.label]['strings'])
            
            # for i, string in enumerate(samples[d.label][s.label]['strings']): 
            #     samples[d.label][s.label]['strings'][i] = string.replace("root://cms-xrd-global.cern.ch/", "root://stormgf2.pi.infn.it/")
            
            # chain[d.label][s.label] = samples[d.label][s.label]['strings']
            if nfiles_max>len(samples[d.label][s.label]['strings']): 
                chain[d.label][s.label] = samples[d.label][s.label]['strings']
            else: 
                chain[d.label][s.label] = samples[d.label][s.label]['strings'][:nfiles_max]
        else: 
            nfiles = nfiles_max
            chain[d.label][s.label] = samples[d.label][s.label]['strings'][:nfiles]
        if not "Data" in s.label: ntot_events[d.label][s.label] = np.sum(samples[d.label][s.label]['ntot'][:nfiles])
        else: ntot_events[d.label][s.label] = None
        print("Dataset : "+s.label)
        print("# of files to process : ", nfiles)
        if distributed and nfiles_max>2:
            print("files strings :\n  {}\n  {}\n  ... \n  {}\n  {}".format(chain[d.label][s.label][0], chain[d.label][s.label][1], chain[d.label][s.label][-2], chain[d.label][s.label][-1]))
        else :
            print("files strings :\n  {}".format(chain[d.label][s.label][0]))
        print("# of total events in the files to process (MC only, if Data the number is None) : ", ntot_events[d.label][s.label])

Regions to book: 
  All
  Presel
  PreselWpt
  1TopLep
  1TopLepWpt
  1TopLep_1TopHadrRes_WPloose
  1TopLep_1TopHadrMix_WPloose
  1TopLep_1TopHadrMer_WPloose
  1TopLep_1TopHadrAll_WPloose
  1TopLep_1TopHadrResWpt_WPloose
  1TopLep_1TopHadrMixWpt_WPloose
  1TopLep_1TopHadrMerWpt_WPloose
  1TopLep_1TopHadrAllWpt_WPloose
  1TopLep_1TopHadrRes_WPmedium
  1TopLep_1TopHadrMix_WPmedium
  1TopLep_1TopHadrMer_WPmedium
  1TopLep_1TopHadrAll_WPmedium
  1TopLep_1TopHadrResWpt_WPmedium
  1TopLep_1TopHadrMixWpt_WPmedium
  1TopLep_1TopHadrMerWpt_WPmedium
  1TopLep_1TopHadrAllWpt_WPmedium
  1TopLep_1TopHadrRes_WPtight
  1TopLep_1TopHadrMix_WPtight
  1TopLep_1TopHadrMer_WPtight
  1TopLep_1TopHadrAll_WPtight
  1TopLep_1TopHadrResWpt_WPtight
  1TopLep_1TopHadrMixWpt_WPtight
  1TopLep_1TopHadrMerWpt_WPtight
  1TopLep_1TopHadrAllWpt_WPtight
Variables for histograms :
['MET_pt', 'MET_phi', 'PuppiMET_pt', 'nGoodJet', 'nJetBtag', 'Top_mass_WPloose', 'Top_pt_WPloose', 'Top_mass_WPmedium', 'Top_pt_WPmedium', 'T

In [6]:
# invLepveto = ""#"!" # per non invertirlo basta passare stinga vuota
# met_cut = 250
# mdphi_cut = 0
# HLT_filter = "HLT_PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60 || HLT_PFMETNoMu120_PFMHTNoMu120_IDTight || HLT_Ele32_WPTight_Gsf || HLT_Ele115_CaloIdVT_GsfTrkIdT || HLT_Photon200 || HLT_IsoMu24"

In [7]:
# initialization of clusters

# upload the proxyfile to the Dask workers to make them able to access data on the grid 

from distributed.diagnostics.plugin import UploadFile
def set_proxy(dask_worker):
    import os
    import shutil
    working_dir = dask_worker.local_directory
    proxy_name = 'x509up_u0'
    os.environ['X509_USER_PROXY'] = working_dir + '/' + proxy_name
    os.environ['X509_CERT_DIR']="/cvmfs/grid.cern.ch/etc/grid-security/certificates/"
    shutil.copyfile(working_dir + '/' + proxy_name, working_dir + '/../../../proxy')    
    os.environ['EXTRA_CLING_ARGS'] = "-O2"
    return os.environ.get("X509_USER_PROXY"), os.environ.get("X509_CERT_DIR")

text_file = open("utils/postselection_tagger.h", "r")
data = text_file.read()
def my_initialization_function():
    print(ROOT.gInterpreter.ProcessLine(".O"))
    ROOT.gInterpreter.Declare('{}'.format(data))
    print("end of initialization")

# set up everything properly
if distributed == True:
    RDataFrame = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame
    client = Client(address="tcp://127.0.0.1:"+str(sched_port))
    client.restart()
    client.register_worker_plugin(UploadFile("/tmp/x509up_u0"))
    client.run(set_proxy)
    ROOT.RDF.Experimental.Distributed.initialize(my_initialization_function)
else:
    RDataFrame = ROOT.RDataFrame
    my_initialization_function()

0
end of initialization


In [8]:
################### utils ###################
def cut_string(cut):
    return cut.replace(" ", "").replace("&&","_").replace(">","_g_").replace(".","_").replace("==","_e_")

############### trigger selection #####################
def trigger_filter(df):
    df_trig = df.Filter("HLT_Mu50 || HLT_IsoMu24 || HLT_OldMu100 || HLT_TkMu100 || HLT_Ele32_WPTight_Gsf", "trigger")
    return df_trig
################### preselection ###############
def goodObjects(df, btagAlg, year, EE):
    df_tmp = df.Define("GoodMu_idx", "GetGoodMuon( Muon_pt,  Muon_eta,  Muon_tightId)")\
               .Define("nVetoEl", "nVetoElectron(Electron_pt, Electron_cutBased, Electron_eta)")\
               .Define("GoodJet_idx", "GetGoodJet(Jet_pt, Jet_jetId, FatJet_eta)")\
               .Define("GoodFatJet_idx", "GetGoodJet(FatJet_pt, FatJet_jetId, FatJet_eta)")\
               .Define("JetBTag_idx", "GetJetBTag(GoodJet_idx, "+bTagAlg+","+str(year)+","+str(EE)+")")\
               .Define("nGoodJet", "static_cast<int>(GoodJet_idx.size());")\
               .Define("nJetBtag", "static_cast<int>(JetBTag_idx.size());")
    return df_tmp
############### top selection ########################
def LeptonTopTag(df, isMC):
    # df_ = df.Filter("atLeast1GoodLep(GoodMu_idx, GoodEl_idx)").Filter("nJetBtag>0")\
    df_ = df.Filter("GoodMu_idx.size()==1").Filter("nJetBtag>0")\
            .Define("Muon_px", "Muon_pt[GoodMu_idx[0]]*sin(Muon_phi[GoodMu_idx[0]])").Define("Muon_py", "Muon_pt[GoodMu_idx[0]]*cos(Muon_phi[GoodMu_idx[0]])")\
            .Define("MET_px", "MET_pt*sin(MET_phi)").Define("MET_py", "MET_pt*cos(MET_phi)")\
            .Define("W_pt", "sqrt(pow(Muon_px+MET_px, 2)+pow(Muon_py+MET_py, 2))")\
            .Define("GoodTopMuBJet_idx","countTopMu(GoodMu_idx, Muon_eta, Muon_phi, JetBTag_idx, Jet_eta, Jet_phi)")\
            .Define("GoodTopMuBJet_dr", "drBMu(GoodMu_idx, Muon_eta, Muon_phi, JetBTag_idx, Jet_eta, Jet_phi)")\
            .Define("NGoodTopMu", "static_cast<int>(GoodTopMuBJet_idx.size());")\
            .Define("BJetTopMu", "nearest(GoodTopMuBJet_idx, GoodTopMuBJet_dr)")
            # .Define("GoodTopLepBJet_idx","countTopLep(GoodMu_idx, Muon_eta, Muon_phi, GoodEl_idx, Electron_eta, Electron_phi, JetBTag_idx, Jet_eta, Jet_phi)")\
            # .Define("GoodTopLepBJet_dr", "drBLep(GoodMu_idx, Muon_eta, Muon_phi, GoodEl_idx, Electron_eta, Electron_phi, JetBTag_idx, Jet_eta, Jet_phi)")\
            # .Define("NGoodTopLep", "static_cast<int>(GoodTopLepBJet_idx.size());")\
            # .Define("BJetTopLep", "nearest(GoodTopLepBJet_idx, GoodTopLepBJet_dr)")
    # if isMC:
    #     df_ = df_.Define("BJetTopLep_MCtag", "BJetTopLep_MCtag(Jet_matched, Jet_pdgId, Jet_topMother, BJetTopLep)")
    return df_

def select_top(df, isMC):

    for top in topTypes:
        if top == "Resolved": idxFatJet = "{}"
        else: idxFatJet = "Top"+top+"_idxFatJet"
        df = df.Define("ClusterT1Top"+top, "findTopClustersType1(Top"+top+"_TopScore, "+idxFatJet+", Top"+top+"_idxJet0, Top"+top+"_idxJet1, Top"+top+"_idxJet2,BJetTopMu)")\
                            .Define("ClusterT2Top"+top, "findTopClustersType2(Top"+top+"_TopScore, "+idxFatJet+", Top"+top+"_idxJet0, Top"+top+"_idxJet1, Top"+top+"_idxJet2,BJetTopMu)")\
                            .Define("nClusterT1Top"+top, "static_cast<int> (ClusterT1Top"+top+".size());")\
                            .Define("nClusterT2Top"+top, "static_cast<int> (ClusterT2Top"+top+".size());")
        for wp in ["WPloose","WPmedium","WPtight"]:
            df = df.Define("ClusterT1Top"+top+"Reco_"+wp, "RecoTaggedTopCluster"+top+"(ClusterT1Top"+top+", Top"+top+"_TopScore, "+Top_threshold[top][wp]+")")\
                   .Define("ClusterT2Top"+top+"Reco_"+wp, "RecoTaggedTopCluster"+top+"(ClusterT2Top"+top+", Top"+top+"_TopScore, "+Top_threshold[top][wp]+")")
            df = df.Define("nTopClusterT1"+top+"Reco_"+wp, "Sum(ClusterT1Top"+top+"Reco_"+wp+">0)")\
                   .Define("nTopClusterT2"+top+"Reco_"+wp, "Sum(ClusterT2Top"+top+"Reco_"+wp+">0)")
    if (isMC):
        for top in topTypes:
            if top == "Resolved": idxFatJet = "{}"
            else: idxFatJet = "Top"+top+"_idxFatJet"
            for tr in TruthType["Top"+top].keys():
                if tr != "standard": df = df.Define(TruthType["Top"+top+""][tr], "CalculateTop"+tr+"(Top"+top+"_idxJet0, Top"+top+"_idxJet1, Top"+top+"_idxJet2, "+idxFatJet+", Jet_matched, Jet_topMother, Jet_pdgId, FatJet_matched, FatJet_topMother, FatJet_pdgId)")
                
                df = df.Define("ClusterT1Top"+top+"MCTag"+tr, "MCTaggedTopCluster(ClusterT1Top"+top+", "+TruthType["Top"+top+""][tr]+")")\
                        .Define("ClusterT2Top"+top+"MCTag"+tr, "MCTaggedTopCluster(ClusterT2Top"+top+", "+TruthType["Top"+top+""][tr]+")")\
                        .Define("nTopClusterT1"+top+"MCTag"+tr, "Sum(ClusterT1Top"+top+"MCTag"+tr+">0)")\
                        .Define("nTopClusterT2"+top+"MCTag"+tr, "Sum(ClusterT2Top"+top+"MCTag"+tr+">0)")\
                        .Define("TopClusterT1"+top+"MCTagTrueFirstCluster"+tr, "ClusterT1Top"+top+"MCTag"+tr+"[0] && ClusterT1Top"+top+"MCTag"+tr+"[0]!=-1")\
                        .Define("TopClusterT2"+top+"MCTagTrueFirstCluster"+tr, "ClusterT2Top"+top+"MCTag"+tr+"[0] && ClusterT2Top"+top+"MCTag"+tr+"[0]!=-1")\
                        .Define("TopClusterT1"+top+"MCTagFalseFirstCluster"+tr, "!ClusterT1Top"+top+"MCTag"+tr+"[0] && ClusterT1Top"+top+"MCTag"+tr+"[0]!=-1")\
                        .Define("TopClusterT2"+top+"MCTagFalseFirstCluster"+tr, "!ClusterT2Top"+top+"MCTag"+tr+"[0] && ClusterT2Top"+top+"MCTag"+tr+"[0]!=-1")
                for wp in ["WPloose","WPmedium","WPtight"]:
                    df = df.Define("nTopClusterT1"+top+"TrueReco"+tr+"_"+wp, "Sum(ClusterT1Top"+top+"MCTag"+tr+">0 && ClusterT1Top"+top+"Reco_"+wp+">0)")\
                            .Define("nTopClusterT2"+top+"TrueReco"+tr+"_"+wp, "Sum(ClusterT2Top"+top+"MCTag"+tr+">0 && ClusterT2Top"+top+"Reco_"+wp+">0)")\
                            .Define("TopClusterT1"+top+"TrueRecoFirstCluster"+tr+"_"+wp, "ClusterT1Top"+top+"MCTag"+tr+"[0] && ClusterT1Top"+top+"Reco_"+wp+"[0] && ClusterT1Top"+top+"Reco_"+wp+"[0]!=-1")\
                            .Define("TopClusterT2"+top+"TrueRecoFirstCluster"+tr+"_"+wp, "ClusterT2Top"+top+"MCTag"+tr+"[0] && ClusterT2Top"+top+"Reco_"+wp+"[0] && ClusterT2Top"+top+"Reco_"+wp+"[0]!=-1")\
                            .Define("TopClusterT1"+top+"FalseRecoFirstCluster"+tr+"_"+wp, "!ClusterT1Top"+top+"MCTag"+tr+"[0] && ClusterT1Top"+top+"Reco_"+wp+"[0] && ClusterT1Top"+top+"Reco_"+wp+"[0] !=-1")\
                            .Define("TopClusterT2"+top+"FalseRecoFirstCluster"+tr+"_"+wp, "!ClusterT2Top"+top+"MCTag"+tr+"[0] && ClusterT2Top"+top+"Reco_"+wp+"[0] && ClusterT2Top"+top+"Reco_"+wp+"[0] !=-1")
        df = df.Define("TopMixed_Score_truthstandard", "TopMixed_TopScore[TopMixed_truth==1]")\
               .Define("TopMixed_Score_truth1", "TopMixed_TopScore[TopMixed_truth1==1 && TopMixed_truth==0 && TopMixed_truth2==0]")\
               .Define("TopMixed_Score_truth2", "TopMixed_TopScore[TopMixed_truth2==1 && TopMixed_truth==0]")\
               .Define("TopMixed_Score_truth1_incl", "TopMixed_TopScore[TopMixed_truth1==1]")\
               .Define("TopMixed_Score_truth2_incl", "TopMixed_TopScore[TopMixed_truth2==1]")\
               .Define("TopResolved_Score_truthstandard", "TopResolved_TopScore[TopResolved_truth==1]")\
               .Define("TopResolved_Score_truth1", "TopResolved_TopScore[TopResolved_truth1==1 && TopResolved_truth==0 && TopResolved_truth2==0]")\
               .Define("TopResolved_Score_truth2", "TopResolved_TopScore[TopResolved_truth2==1 && TopResolved_truth==0]")\
               .Define("TopResolved_Score_truth1_incl", "TopResolved_TopScore[TopResolved_truth1==1]")\
               .Define("TopResolved_Score_truth2_incl", "TopResolved_TopScore[TopResolved_truth2==1]")

    df_topselected = df.Define("Top_idx_Resolved","ClusterT1TopResolved.size()>0 ? ClusterT1TopResolved[0][0]: -1")\
                       .Define("Top_idx_Mixed","ClusterT1TopMixed.size() ? ClusterT1TopMixed[0][0] : -1")
    df_topselected = df_topselected.Define("TopMerged_bestTopScore", "FatJet_particleNet_TvsQCD[ArgMax(FatJet_particleNet_TvsQCD)]")\
                                   .Define("TopMerged_bestTopPt", "FatJet_pt[ArgMax(FatJet_particleNet_TvsQCD)]")\
                                   .Define("TopMerged_bestTopEta", "FatJet_eta[ArgMax(FatJet_particleNet_TvsQCD)]")\
                                   .Define("TopMerged_bestTopPhi", "FatJet_phi[ArgMax(FatJet_particleNet_TvsQCD)]")\
                                   .Define("TopMerged_bestTopMass", "FatJet_mass[ArgMax(FatJet_particleNet_TvsQCD)]")\
                                   .Define("TopMerged_bestTopTruth", "FatJet_matched[ArgMax(FatJet_particleNet_TvsQCD)] == 3")\
                                   .Define("TopMerged_truth_exists", "Any(FatJet_matched==3)")
    
    df_topvariables = df_topselected.Define("Top_pt_Resolved", "Top_idx_Resolved !=-1 ? TopResolved_pt[Top_idx_Resolved]: -999")\
                        .Define("Top_eta_Resolved", "Top_idx_Resolved!=-1 ? TopResolved_eta[Top_idx_Resolved]: -999")\
                        .Define("Top_phi_Resolved", "Top_idx_Resolved!=-1 ? TopResolved_phi[Top_idx_Resolved]: -999")\
                        .Define("Top_mass_Resolved", "Top_idx_Resolved!=-1 ? TopResolved_mass[Top_idx_Resolved]: -999")\
                        .Define("Top_score_Resolved", "Top_idx_Resolved!=-1 ? TopResolved_TopScore[Top_idx_Resolved]: -999")\
                        .Define("Top_idxJet0_Resolved", "Top_idx_Resolved!=-1 ? TopResolved_idxJet0[Top_idx_Resolved]: -999")\
                        .Define("Top_idxJet1_Resolved", "Top_idx_Resolved!=-1 ? TopResolved_idxJet1[Top_idx_Resolved]: -999")\
                        .Define("Top_idxJet2_Resolved", "Top_idx_Resolved!=-1 ? TopResolved_idxJet2[Top_idx_Resolved]: -999")\
                        .Define("Top_pt_Mixed", "Top_idx_Mixed!=-1 ? TopMixed_pt[Top_idx_Mixed]: -999")\
                        .Define("Top_eta_Mixed", "Top_idx_Mixed!=-1 ? TopMixed_eta[Top_idx_Mixed]: -999")\
                        .Define("Top_phi_Mixed", "Top_idx_Mixed!=-1 ? TopMixed_phi[Top_idx_Mixed]: -999")\
                        .Define("Top_mass_Mixed", "Top_idx_Mixed!=-1 ? TopMixed_mass[Top_idx_Mixed]: -999")\
                        .Define("Top_score_Mixed", "Top_idx_Mixed!=-1 ? TopMixed_TopScore[Top_idx_Mixed]: -999")\
                        .Define("Top_idxJet0_Mixed", "Top_idx_Mixed!=-1 ? TopMixed_idxJet0[Top_idx_Mixed]: -999")\
                        .Define("Top_idxJet1_Mixed", "Top_idx_Mixed!=-1 ? TopMixed_idxJet1[Top_idx_Mixed]: -999")\
                        .Define("Top_idxJet2_Mixed", "Top_idx_Mixed!=-1 ? TopMixed_idxJet2[Top_idx_Mixed]: -999")\
                        .Define("Top_idxFatJet_Mixed", "Top_idx_Mixed!=-1 ? TopMixed_idxFatJet[Top_idx_Mixed]: -999")\
    # SELECTION of the best Top among categories
    
    # return indices of the FatJet with particleNet score over the thresholds 
    for wp in ["WPloose","WPmedium","WPtight"]:
        df_topvariables = df_topvariables.Define("GoodTopMer_idx_"+wp, "select_TopMer(FatJet_particleNet_TvsQCD, GoodFatJet_idx, "+TopMerged_threshold[wp]+")")
        # return indices of the TopMixed over the threshold with any object in common
        df_topvariables = df_topvariables.Define("GoodTopMix_idx_"+wp, "select_TopMix(TopMixed_TopScore, TopMixed_idxFatJet, TopMixed_idxJet0, TopMixed_idxJet1, TopMixed_idxJet2, GoodJet_idx, GoodFatJet_idx, BJetTopMu, "+Top_threshold["Mixed"][wp]+")")
        # return indices of the TopResolved over the threshold with any object in common
        df_topvariables = df_topvariables.Define("GoodTopRes_idx_"+wp, "select_TopRes(TopResolved_TopScore, TopResolved_idxJet0, TopResolved_idxJet1, TopResolved_idxJet2, GoodJet_idx, BJetTopMu, "+Top_threshold["Resolved"][wp]+")")
    
        df_topvariables = df_topvariables.Define("nGoodTopResolved_"+wp, "nTop(GoodTopRes_idx_"+wp+")")\
                                .Define("nGoodTopMixed_"+wp, "nTop(GoodTopMix_idx_"+wp+")")\
                                .Define("nGoodTopMerged_"+wp, "nTop(GoodTopMer_idx_"+wp+")")
    
    
        # return:  1- Event Resolved, 2- Event Mixed, 3- Event Merged, 4- Event Nothing, ...
        df_topvariables = df_topvariables.Define("EventTopCategory_"+wp, "select_TopCategory(GoodTopMer_idx_"+wp+", GoodTopMix_idx_"+wp+", GoodTopRes_idx_"+wp+")")
        # if isMC:
        #     df_topcategory = df_topcategory.Define("EventTopCategoryWithTruth", "select_TopCategoryWithTruth(EventTopCategory, FatJet_matched, GoodTopMer_idx, TopMixed_truth, GoodTopMix_idx, TopResolved_truth, GoodTopRes_idx)")
    
        df_topvariables = df_topvariables.Define("Top_idx_"+wp,
                                               "select_bestTop(EventTopCategory_"+wp+", FatJet_particleNet_TvsQCD, TopMixed_TopScore, TopResolved_TopScore)")
        # return best top idx wrt category --> the idx is referred to the list of candidates fixed by the EventTopCategory
        df_topvariables = df_topvariables.Define("Top_pt_"+wp, "select_TopVar(EventTopCategory_"+wp+", Top_idx_"+wp+", FatJet_pt, TopMixed_pt, TopResolved_pt)")\
                            .Define("Top_eta_"+wp, "select_TopVar(EventTopCategory_"+wp+", Top_idx_"+wp+", FatJet_eta, TopMixed_eta, TopResolved_eta)")\
                            .Define("Top_phi_"+wp, "select_TopVar(EventTopCategory_"+wp+", Top_idx_"+wp+", FatJet_phi, TopMixed_phi, TopResolved_phi)")\
                            .Define("Top_mass_"+wp, "select_TopVar(EventTopCategory_"+wp+", Top_idx_"+wp+", FatJet_mass, TopMixed_mass, TopResolved_mass)")\
                            .Define("Top_score_"+wp, "select_TopVar(EventTopCategory_"+wp+", Top_idx_"+wp+", FatJet_particleNet_TvsQCD, TopMixed_TopScore, TopResolved_TopScore)")\


    return df_topvariables

In [9]:
def bookhisto(df, regions_def, var, s_cut):
    h_ = {}
    for reg in regions_def.keys():
        h_[reg] = {}
        for v in var:
            if v._MConly and not sampleflag: 
                continue
            else:
                if regions_def[reg] == "":
                    if v._noUnOvFlowbin:
                        h_[reg][v._name]= df.Histo1D((v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xmin, v._xmax), v._name, "w_nominal")
                    else:
                        h_[reg][v._name]= df.Redefine(v._name, "UnOvBin("+v._name+","+str(v._nbins)+","+str(v._xmin)+","+str(v._xmax)+")").Histo1D((v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xmin, v._xmax), v._name, "w_nominal")
                else:
                    if v._noUnOvFlowbin:
                        h_[reg][v._name]= df.Filter(regions_def[reg]).Histo1D((v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xmin, v._xmax), v._name, "w_nominal")
                    else:
                        h_[reg][v._name]= df.Filter(regions_def[reg]).Redefine(v._name, "UnOvBin("+v._name+","+str(v._nbins)+","+str(v._xmin)+","+str(v._xmax)+")").Histo1D((v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xmin, v._xmax), v._name, "w_nominal")
                    
    return h_

def bookhisto2D(df, regions_def, var2d, s_cut):
    h_ = {}
    for reg in regions_def.keys():
        h_[reg] = {}
        for v in var2d:
            if regions_def[reg]=="":
                h_[reg][v._name] = df.Redefine(v._xname, "UnOvBin("+v._xname+","+str(v._nxbins)+","+str(v._xmin)+","+str(v._xmax)+")")\
                                     .Redefine(v._yname, "UnOvBin("+v._yname+","+str(v._nybins)+","+str(v._ymin)+","+str(v._ymax)+")")\
                                     .Histo2D((v._xname+"Vs"+v._yname+"_"+reg+"_"+s_cut," ;"+v._xtitle+";"+v._ytitle, v._nxbins, v._xmin, v._xmax, v._nybins, v._ymin, v._ymax), v._xname, v._yname)
            else:
                h_[reg][v._name] = df.Filter(regions_def[reg])\
                                     .Redefine(v._xname, "UnOvBin("+v._xname+","+str(v._nxbins)+","+str(v._xmin)+","+str(v._xmax)+")")\
                                     .Redefine(v._yname, "UnOvBin("+v._yname+","+str(v._nybins)+","+str(v._ymin)+","+str(v._ymax)+")")\
                                     .Histo2D((v._xname+"Vs"+v._yname+"_"+reg+"_"+s_cut," ;"+v._xtitle+";"+v._ytitle, v._nxbins, v._xmin, v._xmax, v._nybins, v._ymin, v._ymax), v._xname, v._yname)
    return h_

In [10]:
def savehisto(d, h, regions_def, var, s_cut):
    histo = {reg: {v._name: ROOT.TH1D(v._name+"_"+reg+"_"+s_cut," ;"+v._title+"", v._nbins, v._xmin, v._xmax) for v in var} for reg in regions_def.keys()}
    isMC=True
    if "Data" in d.label: isMC = False
    if hasattr(d, "components"):
        s_list = d.components
    else:
        s_list = [d]
    
    for s in s_list:
        outfile = ROOT.TFile.Open(repohisto+s.label+'.root', "RECREATE")
        for reg in regions_def.keys():
            for v in var:
                if v._MConly and not isMC:
                    continue
                else:
                    # print(reg, v)
                    histo[reg][v._name] = h[d.label][s.label][reg][v._name].GetValue()      
                    outfile.cd()
                    histo[reg][v._name].Write()
        outfile.Close()

# i plot2d per il momento non ci servono, si deve trovare un modo più intelligente di farli
def savehisto2d(d, h, regions_def, var2d, s_cut):
    histo = {reg: {v._name: ROOT.TH2D(v._name+"_"+reg+"_"+s_cut," ;"+v._xtitle+";"+v._ytitle, v._nxbins, v._xmin, v._xmax, v._nybins, v._ymin, v._ymax,) for v in var2d} for reg in regions_def.keys()}
        
    if hasattr(d, "components"):
        s_list = d.components
    else:
        s_list = [d]
    
    for s in s_list:
        outfile = ROOT.TFile.Open(repohisto+s.label+'_2D.root', "RECREATE")
        for reg in regions_def.keys():
            for v in histo[reg].keys():
                histo[reg][v] = h[d.label][s.label][reg][v].GetValue()      
                outfile.cd()
                histo[reg][v].Write()
        outfile.Close()

In [11]:
#samples['DataHTF_2022']['DataHTF_2022']['strings']

In [12]:
#samples[d.label][d.components[0]]['strings']

In [13]:
t0 = datetime.now()
print("starting loop on datasets: ",[d.label for d in datasets])
print("Local time :", t0)
# print("requirements: "+cut)

h = {}
h_2D = {}

workers = {}
cpu_percentages = {}


for d in datasets:
    if hasattr(d, "components"):
        s_list = d.components
    else:
        s_list = [d]

    if 'Data' in d.label : sampleflag = 0
    else: sampleflag = 1
    c_ = cut
    h[d.label] = {}
    h_2D[d.label] = {}
    for s in s_list:
        
#         if distributed:
#             worker_cpu_usage = get_worker_cpu_usage(client)
#             # print("Utilizzo della CPU di ogni worker:", worker_cpu_usage)

#             workers["start"] = list(worker_cpu_usage.keys())
#             cpu_percentages["start"] = list(worker_cpu_usage.values())
        
        #-------------------------------------------------------------------------
        ############# Fixing variables for 2018-2022 #############################
        #-------------------------------------------------------------------------
        if s.year == 2018:
            bTagAlg = "Jet_btagDeepB"
        elif s.year == 2022:
            bTagAlg = "Jet_btagPNetB"
        if hasattr(s,"EE"):
            EE = s.EE
        else:
            EE = 0
        #-------------------------------------------------------------------------
        #########################  DF initialization #############################
        #-------------------------------------------------------------------------
        
        print("Initializing DataFrame for "+ s.label +" chain len = ", len(chain[d.label][s.label]))
        if len(chain[d.label][s.label])==1: print(chain[d.label][s.label])
        if distributed ==True:
            df = RDataFrame("Events", chain[d.label][s.label], npartitions=nmaxpartition, 
                            daskclient=client, monitor_label = "main" )
        else:
            df = RDataFrame("Events", chain[d.label][s.label])
        
        # if 'TT' in s.label and not 'Mtt' in s.label:
        #     df = df.Filter("tt_mtt_doublecounting(GenPart_pdgId, GenPart_pt, GenPart_eta, GenPart_phi, GenPart_mass)", "TT double counting")
        if sampleflag:
            df = df.Define("GenPartTop_pt", "genpartTopPt(GenPart_pt, GenPart_pdgId, GenPart_genPartIdxMother, GenPart_genPartIdxMother_prompt)")
            branches.update(["GenPartTop_pt"])
        
        df_ismc         = df.Define("isMC", "isMC("+str(sampleflag)+")")
        df_year         = df_ismc.Define("year", str(s.year))
        df_hemveto      = df_year.Define("HEMVeto", "hemveto(Jet_eta, Jet_phi, Electron_eta, Electron_phi)")
        df_hemveto      = df_hemveto.Filter("(isMC || (year != 2018) || (HEMVeto || run<319077.))")
        df_hlt          = trigger_filter(df_hemveto)
        df_wnom         = df_hlt.Define('w_nominal', '1').Redefine('w_nominal', 'w_nominalhemveto(w_nominal, HEMVeto, year)')
        df_presel       = goodObjects(df_wnom, bTagAlg, s.year, EE)
        df_LepTop       = LeptonTopTag(df_presel, sampleflag)
        df_TopSel       = select_top(df_LepTop, sampleflag)
        
        df_snap = df_TopSel
        df_plot = df_TopSel
        
        if do_snapshot:
            opts = ROOT.RDF.RSnapshotOptions()
            opts.fLazy = True
            if distributed: fold = "./"
            else: fold = folder
            snapshot_df = df_snap.Snapshot("events_nominal", fold+"snapTagger_"+s.label+".root", branches, opts)
            # print("./"+s.label+".root")
        if do_histos:
            s_cut = cut_string(cut)
            if len(var) != 0 :
                h[d.label][s.label] = bookhisto(df_plot, regions_def, var, s_cut)
            if len(var2d) != 0 :
                h_2D[d.label][s.label] = bookhisto2D(df_plot, regions_def, var2d, s_cut)
# if not distributed:
#     df_presel.Report().Print()
# if distributed:
#     worker_cpu_usage = get_worker_cpu_usage(client)
#     # print("Utilizzo della CPU di ogni worker:", worker_cpu_usage)

#     workers["after_book_hist"] = list(worker_cpu_usage.keys())
#     cpu_percentages["after_book_hist"] = list(worker_cpu_usage.values())
if do_histos:
    print("All histos booked !")
    for d in datasets:
        if len(var):
            savehisto(d, h, regions_def, var, s_cut)
        if len(var2d) != 0 :
            savehisto2d(d, h_2D, regions_def, var2d, s_cut)
        print(d.label + " histos saved")
# if distributed:
#     worker_cpu_usage = get_worker_cpu_usage(client)
#     # print("Utilizzo della CPU di ogni worker:", worker_cpu_usage)

#     workers["end"] = list(worker_cpu_usage.keys())
#     cpu_percentages["end"] = list(worker_cpu_usage.values())
    
if do_snapshot:
    snapshot_df.GetValue()
    if distributed: 
        client.run(transfer_to_tier)
        print("Snapshots saved and trasfered to tier")
    print("Sanpshot done!")
t1 = datetime.now()
print("Job finished in: ", t1-t0)

starting loop on datasets:  ['WJetsHT2500toInf_2018']
Local time : 2024-06-21 12:26:21.779792
Initializing DataFrame for WJetsHT2500toInf_2018 chain len =  1
['root://cms-xrd-global.cern.ch//store/user/acagnott/DM_Run3_v0/WJetsToLNu_HT-2500ToInf_TuneCP5_13TeV-madgraphMLM-pythia8/WJetsHT2500toInf_2018/231222_110712/0000/tree_hadd_23.root\n']
All histos booked !
WJetsHT2500toInf_2018 histos saved
Sanpshot done!
Job finished in:  0:11:50.367933


Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


In [14]:
file = ROOT.TFile.Open(repohisto+s.label+".root")
print(file)
# for a in file.GetListOfKeys(): print(a)
for reg in regions_def.keys():
    # for v in var:
    hist = file.Get(var[0]._name+"_"+reg+"_")
    print(reg, hist.Integral())

Name: ./results/TopTagger_JetMET_selectioncomparison/plots/WJetsHT2500toInf_2018.root Title: 
All 136.0
Presel 120.0
PreselWpt 76.0
1TopLep 58.0
1TopLepWpt 40.0
1TopLep_1TopHadrRes_WPloose 0.0
1TopLep_1TopHadrMix_WPloose 24.0
1TopLep_1TopHadrMer_WPloose 5.0
1TopLep_1TopHadrAll_WPloose 29.0
1TopLep_1TopHadrResWpt_WPloose 0.0
1TopLep_1TopHadrMixWpt_WPloose 15.0
1TopLep_1TopHadrMerWpt_WPloose 4.0
1TopLep_1TopHadrAllWpt_WPloose 19.0
1TopLep_1TopHadrRes_WPmedium 0.0
1TopLep_1TopHadrMix_WPmedium 2.0
1TopLep_1TopHadrMer_WPmedium 4.0
1TopLep_1TopHadrAll_WPmedium 6.0
1TopLep_1TopHadrResWpt_WPmedium 0.0
1TopLep_1TopHadrMixWpt_WPmedium 1.0
1TopLep_1TopHadrMerWpt_WPmedium 3.0
1TopLep_1TopHadrAllWpt_WPmedium 4.0
1TopLep_1TopHadrRes_WPtight 0.0
1TopLep_1TopHadrMix_WPtight 0.0
1TopLep_1TopHadrMer_WPtight 3.0
1TopLep_1TopHadrAll_WPtight 3.0
1TopLep_1TopHadrResWpt_WPtight 0.0
1TopLep_1TopHadrMixWpt_WPtight 0.0
1TopLep_1TopHadrMerWpt_WPtight 2.0
1TopLep_1TopHadrAllWpt_WPtight 2.0


In [15]:
# regions_def.keys()

In [16]:
# region = "NoCut"
# var = "MET_pt"

In [17]:
if do_snapshot and distributed:
    print(os.popen("davix-ls davs://stwebdav.pi.infn.it:8443/cms/store/user/acagnott/{}/{}/ -E /tmp/x509up_u0 --capath /cvmfs/cms.cern.ch/grid/etc/grid-security/certificates/".format(remote_folder_name, remote_subfolder_name)).read())

In [18]:
c1 = []
hCPU = []
for phase in workers.keys():
    hCPU.append(ROOT.TH1F("hCPU"+phase,"workers CPU usage "+phase+";workers;CPU usage (%)", len(workers[phase]), 0.5, len(workers[phase])+0.5))
    for i in range(len(workers[phase])):
        hCPU[-1].SetBinContent(i+1, cpu_percentages[phase][i])
    c1.append(ROOT.TCanvas())
    c1[-1].Draw()
    hCPU[-1].Draw("HIST")